In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import numpy as np
import pandas as pd
from params import *

from tqdm import tqdm
from itertools import combinations
from utils.read2df import read2df
from utils.cointncorr import CointnCorr

In [2]:
# %%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

Found 4 symbols
[1/4] - start download monthly EURUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-07.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-08.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/EURUSDT/5m/EURUSDT-5m-2017-09.zip



In [3]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

# have a preview
dfs[0]

,time,open,high,low,close,volume,tic,itvl,datetime
0,1595246699999,1.1383,1.1393,1.1383,1.1393,5250.60,EURUSDT,5m,2020-07-20 12:04:59.999
1,1595246699999,1.2480,2.7926,1.0174,1.2625,19670.50,GBPUSDT,5m,2020-07-20 12:04:59.999
2,1595246999999,1.1385,1.1392,1.1383,1.1388,4747.88,EURUSDT,5m,2020-07-20 12:09:59.999
3,1595246999999,1.2625,1.3445,1.2499,1.2501,6355.17,GBPUSDT,5m,2020-07-20 12:09:59.999
4,1595247299999,1.1390,1.1391,1.1390,1.1390,3494.37,EURUSDT,5m,2020-07-20 12:14:59.999
...,...,...,...,...,...,...,...,...,...
689755,1698796199999,1.2370,1.2370,1.2370,1.2370,530.00,GBPUSDT,5m,2023-10-31 23:49:59.999
689756,1698796499999,1.0564,1.0565,1.0564,1.0564,16683.30,EURUSDT,5m,2023-10-31 23:54:59.999
689757,1698796499999,1.2370,1.2370,1.2370,1.2370,32.10,GBPUSDT,5m,2023-10-31 23:54:59.999
689758,1698796799999,1.0564,1.0565,1.0564,1.0564,34455.50,EURUSDT,5m,2023-10-31 23:59:59.999


In [4]:
'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []
for i in range(len(dfs)):
    trains.append(dfs[i][(dfs[i]['datetime'] > start_date) & (dfs[i]['datetime'] < trade_date)].reset_index(drop=True))
    tests.append(dfs[i][dfs[i]['datetime'] >= trade_date].reset_index(drop=True))

In [5]:
'''
Find the best pair
'''

import pickle

tables = CointnCorr(trains, freqs).tabulate()
cointncorrtxt = f"result/gridsearch/cointncorr.txt"

os.remove(f"{cointncorrtxt}") if os.path.exists(f"{cointncorrtxt}") else None

with open(cointncorrtxt, "w+") as f:
    for k, v in tables.items():
        f.write(f"{k}\n")
        f.write(f"{v}\n\n")
f.close()

best_value = 0
for key in tables.keys():
    for freq in freqs:
        rel = tables[key].at['coint', freq] + tables[key].at['corr', freq]
        if rel > best_value:
            best_value = rel
            best_pair = key
            best_freq = freq

print("===========================================")
print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
print("===========================================")

cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)

Best trading pairs shall be: EURUSDT_GBPUSDT under 5m interval
the coint is 35.27%
and the corr is 0.5
